In [1]:
import sys
sys.path.append('../build/')
%pylab inline
np.set_printoptions(precision=4, suppress=True)
import pandas as pd
import linear_solver
import versor as vsr

Populating the interactive namespace from numpy and matplotlib


In [2]:
# import autograd.numpy as np

from pymanopt.manifolds import manifold
from pymanopt.manifolds import Sphere
from pymanopt import Problem
from pymanopt.solvers import SteepestDescent, TrustRegions, ParticleSwarm, ConjugateGradient, NelderMead
from __future__ import division

import numpy as np
import numpy.linalg as la
import numpy.random as rnd

from pymanopt.manifolds.manifold import Manifold

In [102]:
def create_rotor(th_lims=(0, np.pi/2)):
    rotor = vsr.Rot(vsr.Biv(*np.random.uniform(-1, 1, 3)).unit()
                      * np.random.uniform(*th_lims) * -0.5)
    return rotor

In [104]:
def Hessi(A, B, M):
    H = np.zeros((6,6))
    
    MAM = A.spin(M)
    
    a0,a1,a2,a3,a4 = np.array(MAM)
    b0,b1,b2,b3,b4 = np.array(B)
    
    H[0,0] = 4.0 * a1 * b1 + 4.0 * a0 * b0
    H[0,1] = 2.0 * a1 * b2 + 2.0 * a2 * b1
    H[0,2] = -2.0 * a0 * b2 - 2.0 * a2 * b0
    H[0,3] = -2.0 * b1 - 2.0 * a1
    H[0,4] = 2.0 * b0 + 2.0 * a0
    H[1,0] = H[0,1]
    H[2,0] = H[0,2]
    H[3,0] = H[0,3]
    H[4,0] = H[0,4]

    
    H[1,1] = 4.0 * a2 * b2 + 4.0 * a0 * b0
    H[1,2] = 2.0 * a0 * b1 + 2.0 * a1 * b0
    H[1,3] = -2.0 * b2 - 2.0 * a2
    H[1,5] = 2.0 * b0 + 2.0 * a0
    H[2,1] = H[1,2]
    H[3,1] = H[1,3]
    H[5,1] = H[1,5]
    
    H[2,2] = 4.0 * a2 * b2 + 4.0 * a1 * b1
    H[2,4] = -2.0 * b2 - 2.0 * a2
    H[2,5] = 2.0 * b1 + 2.0 * a1
    H[4,2] = H[2,4]
    H[5,2] = H[2,5]

    H[3,3] = 4.0
    H[4,4] = 4.0
    H[5,5] = 4.0
    
    return H

In [105]:
def Hess(points_a, points_b, mot):
    H = np.zeros((6,6))
    for A, B in zip(points_a, points_b):
        H += Hessi(A,B,mot)
    return H

In [273]:
class Motor(Manifold):
    def __init__(self):
        self._shape = 8
        self._name = "Motor manifold"

    def __str__(self):
        return self._name
    
    @property
    def typicaldist(self):
        return 0.5
    
    
    def norm(self, X, U):
        return la.norm(U)
    
    def retr(self, X, U):
        Y = vsr.Mot(*X) + vsr.Mot(*U)
        return np.array(Y.retract())
    
    def CayleySelig(self, X, U):
        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        
        Rp = vsr.Mot(1.0, B[1], B[2], B[3], 0.0, 0.0, 0.0, 0.0)
        Rn = vsr.Mot(1.0, -B[1], -B[2], -B[3], 0.0, 0.0, 0.0, 0.0)
        Rninv = Rn.inv()
        eps = vsr.Mot(0,0,0,0,0,0,0,-1)
        b = vsr.Mot(0.0, B[6], -B[5], B[4], 0.0, 0.0, 0.0, 0.0)
        return np.array(M * (Rp * Rninv + eps * Rninv * b * Rninv * 2))

    def oexp(self,X,U):

        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        n = np.sqrt(1 + B[1] * B[1] + B[2] * B[2] + B[3] * B[3])
        s = B[1] * B[6] - B[2] * B[5] + B[3] * B[4]
        m = vsr.Mot(1.0, B[1], B[2], B[3], B[4], B[5], B[6], s) * (1.0 / n)
        return np.array(M * m)

    def CayleyLi(self,X, U):        
        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        B_ = vsr.Mot(0.0, B[1], B[2], B[3], B[4], B[5], B[6], 0.0)
        BB = B_ * B_
        Rp = vsr.Mot(1.0, B[1], B[2], B[3], B[4], B[5], B[6], 0.0)
        R0 = vsr.Mot(1.0 - BB[0], 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
        R4 = vsr.Mot(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, BB[7])
        Rn = R0 + R4
        Rden = R0 * R0 
        return np.array(M * (Rp * Rp * Rn * Rden.inv()))

    def exp(self, X, U):
        M = vsr.Mot(*X)
        B = M.rev() * vsr.Mot(*U)
        return np.array(M * vsr.Dll(B[1], B[2], B[3], B[4], B[5], B[6]).exp())
    
#     retr = CayleySelig
#     retr = oexp
#     retr = CayleyLi
    retr = exp
    
    def inner(self, X, U, V):
        return float(np.tensordot(U, V, axes=U.ndim))

    def norm(self, X, U):
#         raise RuntimeError('norm')
        return la.norm(U)

    def dist(self, U, V):
        # Make sure inner product is between -1 and 1
        inner = max(min(self.inner(None, U, V), 1), -1)
        return np.arccos(inner)

    def proj(self, X, H):
        M = vsr.Mot(*X)
        B = vsr.Mot(*H)
        return np.array(vsr.Mot(vsr.CGA(M) * grade2(vsr.CGA(M.rev() * B))))
        
    egrad2rgrad = proj

    def ehess2rhess(self, X, egrad, ehess, U):
        return self.proj(X, ehess) - self.inner(None, X, egrad) * U


    @property
    def dim(self):
        return 6

In [274]:
class Rotator(Sphere):
    def __init__(self):
        super().__init__(4)
        
    def exp(self, X, U):
        R = vsr.Rot(*X)
        g = R.rev() * vsr.Rot(*U)
        B = vsr.Biv(g[1], g[2], g[3]) * -1.0
        return np.array(R * B.exp())
    
    def cay(self, X, U):
        R = vsr.Rot(*X)
        g = R.rev() * vsr.Rot(*U)
        Y = vsr.Rot(*self._normalize(np.array([1.0, g[1], g[2], g[3]])))
        return np.array(R * Y)
    
    retr = cay
        
        

In [275]:
def create_motor(d_lims=(0, 1), th_lims=(0, np.pi)):
    translator = (vsr.Vec(*np.random.random(3)).unit()
                  * np.random.uniform(*d_lims)).trs()
    rotator = vsr.Rot(vsr.Biv(*np.random.uniform(-1, 1, 3)).unit()
                      * np.random.uniform(*th_lims) * -0.5)
    motor = translator * rotator
    return motor

In [316]:
n_points=10

m0 = create_motor(th_lims=(0,np.pi/3))
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]

In [317]:
def grade2(Y):
    Y[0] = 0.0
    Y[26] = 0.0
    Y[27] = 0.0
    Y[28] = 0.0
    Y[29] = 0.0
    Y[30] = 0.0
    return Y

In [318]:
def proj(X):
    M = vsr.Mot(0,0,0,0,0,0,0,0)
    sgns = [1,2,3,7]
    for i in range(8):
        Ei = vsr.MotRec(0,0,0,0,0,0,0,0)
        if i in sgns:
            Ei[i] = -1.0
        else:
            Ei[i] = 1.0
            
        Ei[i] = 1.0
        M[i] = (X * vsr.CGA(Ei))[0]
    return vsr.CGA(M)

In [345]:
def adproj(X):
    M = vsr.MotRec(0,0,0,0,0,0,0,0)
    sgns = [1,2,3,7]
    for i in range(8):
        Ei = vsr.Mot(0,0,0,0,0,0,0,0)
        if i in sgns:
            Ei[i] = -1.0
        else:
            Ei[i] = 1.0
            
        M[i] = (vsr.CGA(Ei) * X)[0]
    return vsr.CGA(M)

In [346]:
M = vsr.CGA(create_motor())

In [352]:
A = vsr.CGA(points_a[0])
B = vsr.CGA(points_b[0])
Y = adproj(grade2((A * M.rev() * B + A.rev() * M.rev() * B.rev()) * M.rev()))
print(Y)
G = np.array([(Y * vsr.CGA(vsr.Mot(0,1,0,0,0,0,0,0)))[0],
              (Y * vsr.CGA(vsr.Mot(0,0,1,0,0,0,0,0)))[0],
              (Y * vsr.CGA(vsr.Mot(0,0,0,1,0,0,0,0)))[0],
              (Y * vsr.CGA(vsr.Mot(0,0,0,0,1,0,0,0)))[0],
              (Y * vsr.CGA(vsr.Mot(0,0,0,0,0,1,0,0)))[0],
              (Y * vsr.CGA(vsr.Mot(0,0,0,0,0,0,1,0)))[0]]).reshape(6)
G

CGA: [ 0 0 0 0 0 0 -0.14 0.11 0.033 0.03 0.18 -0.14 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]


array([ 0.1369, -0.1119, -0.0327,  0.0296,  0.1777, -0.1393])

In [356]:
adproj(grade2(A * M.rev() * B * M) * M.rev() * -2)

CGA: [ -0.019 0 0 0 0 0 0.13 -0.11 -0.015 -0.035 -0.18 0.14 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -0.0032 0 0 0 0 0 ]

In [354]:
vsr.Mot(*gradk3(points_a[0], points_b[0], np.array(vsr.Mot(M))))

Mot: [ -0.0036 -0.13 0.1 -0.0046 -0.045 -0.17 0.14 0.018 ]

In [355]:
np.array(vsr.Mot(proj((vsr.CGA(vsr.Mot(*gradk3(points_a[0], points_b[0], np.array(vsr.Mot(M)))))))))

array([-0.0036,  0.13  , -0.1023,  0.0046, -0.0451, -0.1714,  0.137 ,
       -0.0176])

In [283]:
def rotor_rotate_vector(a, X):
    a1, a2, a3 = np.array(a)
    r0, r1, r2, r3 = np.array(X)
    return (a1 * r3 * r3 + (2.0 * a3 * r1 - 2.0 * a2 * r2) * r3 - a1 * r2 * r2 + 2.0 * a3 * r0 * r2 - a1 * r1 * r1 + 2.0 * a2 * r0 * r1 + a1 * r0 * r0,
            (-a2) * r3 * r3 + (2.0 * a3 * r0 - 2.0 * a1 * r2) * r3 + a2 * r2 * r2 - 2.0 * a3 * r1 * r2 - a2 * r1 * r1 - 2.0 * a1 * r0 * r1 + a2 * r0 * r0,
            (-a3) * r3 * r3 + (2.0 * a1 * r1 - 2.0 * a2 * r0) * r3 - a3 * r2 * r2 + (-2.0 * a2 * r1 - 2.0 * a1 * r0) * r2 + a3 * r1 * r1 + a3 * r0 * r0)

In [284]:
def gradk(a, b, X):
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = np.array(X)
    return np.array(vsr.Rot(r0, r1, r2, r3) * vsr.Biv(
        (-2.0 * a1 * b2 - 2.0 * a2 * b1) * r3 * r3 + ((4.0 * a2 * b2 - 4.0 * a1 * b1) * r2 - 4.0 * a2 * b3 * r1 - 4.0 * a1 * b3 * r0) * r3 + (2.0 * a1 * b2 + 2.0 * a2 * b1) * r2 * r2 + (4.0 * a2 * b3 * r0 - 4.0 * a1 * b3 * r1) * r2 + (2.0 * a2 * b1 - 2.0 * a1 * b2) * r1 * r1 + (4.0 * a2 * b2 + 4.0 * a1 * b1) * r0 * r1 + (2.0 * a1 * b2 - 2.0 * a2 * b1) * r0 * r0,
        (-2.0 * a1 * b3 - 2.0 * a3 * b1) * r3 * r3 + (4.0 * a3 * b2 * r2 + (4.0 * a1 * b1 - 4.0 * a3 * b3) * r1 + 4.0 * a1 * b2 * r0) * r3 + (2.0 * a3 * b1 - 2.0 * a1 * b3) * r2 * r2 + ((4.0 * a3 * b3 + 4.0 * a1 * b1) * r0 - 4.0 * a1 * b2 * r1) * r2 + (2.0 * a1 * b3 + 2.0 * a3 * b1) * r1 * r1 + 4.0 * a3 * b2 * r0 * r1 + (2.0 * a1 * b3 - 2.0 * a3 * b1) * r0 * r0,
        (2.0 * a3 * b2 - 2.0 * a2 * b3) * r3 * r3 + (4.0 * a3 * b1 * r2 + 4.0 * a2 * b1 * r1 + (4.0 * a3 * b3 + 4.0 * a2 * b2) * r0) * r3 + (-2.0 * a2 * b3 - 2.0 * a3 * b2) * r2 * r2 + ((4.0 * a3 * b3 - 4.0 * a2 * b2) * r1 + 4.0 * a2 * b1 * r0) * r2 + (2.0 * a2 * b3 + 2.0 * a3 * b2) * r1 * r1 - 4.0 * a3 * b1 * r0 * r1 + (2.0 * a2 * b3 - 2.0 * a3 * b2) * r0 * r0))

In [285]:
def grad_explicit(X): 
    g = np.sum([gradk(a, b, X) for a, b in zip(points_a, points_b)], axis=0) * 2.0
    return g

In [286]:
def costk3(a, b, X):
    a1, a2, a3, a4, a5 = np.array(a)
    b1, b2, b3, b4, b5 = np.array(b)
    m1, m2, m3, m4, m5, m6, m7, m8 = X
    return 4.0 * a4 * b4 * m8 * m8 + ((4.0 * a4 * b1 - 4.0 * a1 * b4) * m4 + (4.0 * a2 * b4 - 4.0 * a4 * b2) * m3 + (4.0 * a4 * b3 - 4.0 * a3 * b4) * m2) * m8 + 4.0 * a4 * b4 * m7 * m7 + ((4.0 * a2 * b4 + 4.0 * a4 * b2) * m4 + (4.0 * a1 * b4 + 4.0 * a4 * b1) * m3 + (4.0 * a4 * b3 - 4.0 * a3 * b4) * m1) * m7 + 4.0 * a4 * b4 * m6 * m6 + ((-4.0 * a3 * b4 - 4.0 * a4 * b3) * m4 + (4.0 * a1 * b4 + 4.0 * a4 * b1) * m2 + (4.0 * a4 * b2 - 4.0 * a2 * b4) * m1) * m6 + 4.0 * a4 * b4 * m5 * m5 + ((-4.0 * a3 * b4 - 4.0 * a4 * b3) * m3 + (-4.0 * a2 * b4 - 4.0 * a4 * b2) * m2 + (4.0 * a4 * b1 - 4.0 * a1 * b4) * m1) * m5 + (2.0 * a4 * b5 + 2.0 * a5 * b4 + 2.0 * a3 * b3 + 2.0 * a2 * b2 - 2.0 * a1 * b1) * m4 * m4 + ((4.0 * a1 * b2 + 4.0 * a2 * b1) * m3 + (-4.0 * a1 * b3 - 4.0 * a3 * b1) * m2 + (4.0 * a2 * b3 - 4.0 * a3 * b2) * m1) * m4 + (2.0 * a4 * b5 + 2.0 * a5 * b4 + 2.0 * a3 * b3 - 2.0 * a2 * b2 + 2.0 * a1 * b1) * m3 * m3 + ((4.0 * a2 * b3 + 4.0 * a3 * b2) * m2 + (4.0 * a1 * b3 - 4.0 * a3 * b1) * m1) * m3 + (2.0 * a4 * b5 + 2.0 * a5 * b4 - 2.0 * a3 * b3 + 2.0 * a2 * b2 + 2.0 * a1 * b1) * m2 * m2 + (4.0 * a1 * b2 - 4.0 * a2 * b1) * m1 * m2 + (2.0 * a4 * b5 + 2.0 * a5 * b4 - 2.0 * a3 * b3 - 2.0 * a2 * b2 - 2.0 * a1 * b1) * m1 * m1

In [287]:
def cost3(X): 
    return np.sum([costk3(a, b, X) for a, b in zip(points_a, points_b)])

In [288]:
def gradk3(a,b,X):
    a1, a2, a3, a4, a5 = np.array(a)
    b1, b2, b3, b4, b5 = np.array(b)
    m1, m2, m3, m4, m5, m6, m7, m8 = X
    B0 = (-4.0 * a2 * b4 * m4 - 4.0 * a1 * b4 * m3) * m8 + (4.0 * a2 * b4 * m3 - 4.0 * a1 * b4 * m4) * m7 + (4.0 * a2 * b4 * m2 + 4.0 * a1 * b4 * m1) * m6 + (4.0 * a1 * b4 * m2 - 4.0 * a2 * b4 * m1) * m5 + (-2.0 * a1 * b2 - 2.0 * a2 * b1) * m4 * m4 + ((4.0 * a2 * b2 - 4.0 * a1 * b1) * m3 - 4.0 * a2 * b3 * m2 - 4.0 * a1 * b3 * m1) * m4 + (2.0 * a1 * b2 + 2.0 * a2 * b1) * m3 * m3 + (4.0 * a2 * b3 * m1 - 4.0 * a1 * b3 * m2) * m3 + (2.0 * a2 * b1 - 2.0 * a1 * b2) * m2 * m2 + (4.0 * a2 * b2 + 4.0 * a1 * b1) * m1 * m2 + (2.0 * a1 * b2 - 2.0 * a2 * b1) * m1 * m1
    B1 = (4.0 * a1 * b4 * m2 - 4.0 * a3 * b4 * m4) * m8 + (4.0 * a3 * b4 * m3 + 4.0 * a1 * b4 * m1) * m7 + (4.0 * a1 * b4 * m4 + 4.0 * a3 * b4 * m2) * m6 + (4.0 * a1 * b4 * m3 - 4.0 * a3 * b4 * m1) * m5 + (-2.0 * a1 * b3 - 2.0 * a3 * b1) * m4 * m4 + (4.0 * a3 * b2 * m3 + (4.0 * a1 * b1 - 4.0 * a3 * b3) * m2 + 4.0 * a1 * b2 * m1) * m4 + (2.0 * a3 * b1 - 2.0 * a1 * b3) * m3 * m3 + ((4.0 * a3 * b3 + 4.0 * a1 * b1) * m1 - 4.0 * a1 * b2 * m2) * m3 + (2.0 * a1 * b3 + 2.0 * a3 * b1) * m2 * m2 + 4.0 * a3 * b2 * m1 * m2 + (2.0 * a1 * b3 - 2.0 * a3 * b1) * m1 * m1
    B2 = (4.0 * a3 * b4 * m3 + 4.0 * a2 * b4 * m2) * m8 + (4.0 * a3 * b4 * m4 + 4.0 * a2 * b4 * m1) * m7 + (4.0 * a2 * b4 * m4 - 4.0 * a3 * b4 * m1) * m6 + (4.0 * a2 * b4 * m3 - 4.0 * a3 * b4 * m2) * m5 + (2.0 * a3 * b2 - 2.0 * a2 * b3) * m4 * m4 + (4.0 * a3 * b1 * m3 + 4.0 * a2 * b1 * m2 + (4.0 * a3 * b3 + 4.0 * a2 * b2) * m1) * m4 + (-2.0 * a2 * b3 - 2.0 * a3 * b2) * m3 * m3 + ((4.0 * a3 * b3 - 4.0 * a2 * b2) * m2 + 4.0 * a2 * b1 * m1) * m3 + (2.0 * a2 * b3 + 2.0 * a3 * b2) * m2 * m2 - 4.0 * a3 * b1 * m1 * m2 + (2.0 * a2 * b3 - 2.0 * a3 * b2) * m1 * m1
    B3 = 4.0 * a4 * b4 * m4 * m8 - 4.0 * a4 * b4 * m3 * m7 - 4.0 * a4 * b4 * m2 * m6 + 4.0 * a4 * b4 * m1 * m5 + (2.0 * a4 * b1 - 2.0 * a1 * b4) * m4 * m4 + (4.0 * a4 * b3 * m2 - 4.0 * a4 * b2 * m3) * m4 + (-2.0 * a1 * b4 - 2.0 * a4 * b1) * m3 * m3 - 4.0 * a4 * b3 * m1 * m3 + (-2.0 * a1 * b4 - 2.0 * a4 * b1) * m2 * m2 - 4.0 * a4 * b2 * m1 * m2 + (2.0 * a4 * b1 - 2.0 * a1 * b4) * m1 * m1
    B4 = -4.0 * a4 * b4 * m3 * m8 - 4.0 * a4 * b4 * m4 * m7 + 4.0 * a4 * b4 * m1 * m6 + 4.0 * a4 * b4 * m2 * m5 + (-2.0 * a2 * b4 - 2.0 * a4 * b2) * m4 * m4 + (-4.0 * a4 * b1 * m3 - 4.0 * a4 * b3 * m1) * m4 + (2.0 * a4 * b2 - 2.0 * a2 * b4) * m3 * m3 - 4.0 * a4 * b3 * m2 * m3 + (-2.0 * a2 * b4 - 2.0 * a4 * b2) * m2 * m2 + 4.0 * a4 * b1 * m1 * m2 + (2.0 * a4 * b2 - 2.0 * a2 * b4) * m1 * m1
    B5 = 4.0 * a4 * b4 * m2 * m8 + 4.0 * a4 * b4 * m1 * m7 + 4.0 * a4 * b4 * m4 * m6 + 4.0 * a4 * b4 * m3 * m5 + (-2.0 * a3 * b4 - 2.0 * a4 * b3) * m4 * m4 + (4.0 * a4 * b1 * m2 + 4.0 * a4 * b2 * m1) * m4 + (-2.0 * a3 * b4 - 2.0 * a4 * b3) * m3 * m3 + (4.0 * a4 * b1 * m1 - 4.0 * a4 * b2 * m2) * m3 + (2.0 * a4 * b3 - 2.0 * a3 * b4) * m2 * m2 + (2.0 * a4 * b3 - 2.0 * a3 * b4) * m1 * m1
    return np.array(vsr.Mot(m1, m2, m3, m4, m5, m6, m7, m8) * vsr.Dll(B0, B1, B2, B3, B4, B5))

In [289]:
def grad_explicit3(X): 
    g = np.sum([gradk3(a, b, X) for a, b in zip(points_a, points_b)], axis=0) * 2.0
    return g

In [290]:
def costk(a, b, X): 
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = X
    o0 = a1 * r3 * r3 + (2.0 * a3 * r1 - 2.0 * a2 * r2) * r3 - a1 * r2 * r2 + 2.0 * a3 * r0 * r2 - a1 * r1 * r1 + 2.0 * a2 * r0 * r1 + a1 * r0 * r0 - b1
    o1 = (-a2) * r3 * r3 + (2.0 * a3 * r0 - 2.0 * a1 * r2) * r3 + a2 * r2 * r2 - 2.0 * a3 * r1 * r2 - a2 * r1 * r1 - 2.0 * a1 * r0 * r1 + a2 * r0 * r0 - b2
    o2 = (-a3) * r3 * r3 + (2.0 * a1 * r1 - 2.0 * a2 * r0) * r3 - a3 * r2 * r2 + (-2.0 * a2 * r1 - 2.0 * a1 * r0) * r2 + a3 * r1 * r1 + a3 * r0 * r0 - b3
    return o0 * o0 + o1 * o1 + o2 * o2

def costk2(a, b, X):
    a1, a2, a3 = np.array(a)
    b1, b2, b3 = np.array(b)
    r0, r1, r2, r3 = X
    return ((a3 * b3 + a2 * b2 - a1 * b1) * r3 * r3 +
            ((2.0 * a1 * b2 + 2.0 * a2 * b1) * r2 + 
             (-2.0 * a1 * b3 - 2.0 * a3 * b1) * r1 + 
             (2.0 * a2 * b3 - 2.0 * a3 * b2) * r0) * r3 + 
            (a3 * b3 - a2 * b2 + a1 * b1) * r2 * r2 + 
            ((2.0 * a2 * b3 + 2.0 * a3 * b2) * r1 + 
             (2.0 * a1 * b3 - 2.0 * a3 * b1) * r0) * r2 + 
            ((-a3) * b3 + a2 * b2 + a1 * b1) * r1 * r1 + 
            (2.0 * a1 * b2 - 2.0 * a2 * b1) * r0 * r1 + 
            ((-a3) * b3 - a2 * b2 - a1 * b1) * r0 * r0)

# (2) Define the cost function (here using autograd.numpy)
def cost(X): 
    return np.sum([costk(a, b, X) for a, b in zip(points_a, points_b)])

def cost2(X): 
    return np.sum([costk2(a, b, X) for a, b in zip(points_a, points_b)])

In [291]:
n_points=3

m0 = create_motor()
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]
manifold = Motor()
manifold.retr = manifold.exp
problem = Problem(manifold=manifold, cost=cost3, grad=grad_explicit3, hess=hess_explicit, verbosity=1)
# problem = Problem(manifold=manifold, cost=cost3, verbosity=1)

# (3) Instantiate a Pymanopt solver
# solver = SteepestDescent()
solver = TrustRegions()
# solver = ConjugateGradient()


# let Pymanopt do the rest
Xopt = solver.solve(problem, x=np.array(create_motor()))
print(m0)
print(vsr.Mot(*Xopt))
print(la.norm(vsr.Mot(*Xopt).rev() * m0))

Compiling cost function...
Optimizing...
 +++ Detected many consecutive TR- (radius decreases).
 +++ Consider decreasing options.Delta_bar by an order of magnitude.
 +++ Current values: Delta_bar = 0.5 and Delta0 = 0.0625
Terminated - max iterations reached after 0.38 seconds.

Mot: [ 0.82 -0.38 -0.4 -0.16 -0.15 -0.015 0.022 0.011 ]
Mot: [ 0.82 -0.38 -0.4 -0.16 -0.15 -0.014 0.021 0.011 ]
1.00000045196


In [292]:
def jaci2(a, mot):
    j = np.zeros((3,6))
    a = a.spin(mot)
    
    j[0,0] = -2 * a[1]
    j[1,0] = 2 * a[0]
    
    j[0,1] = -2 * a[2]
    j[2,1] =  2 * a[0]
    
    j[1,2] = -2 * a[2]
    j[2,2] = 2 * a[1]
    
    j[0,3] = 2
    j[1,4] = 2
    j[2,5] = 2
     
    return j

In [293]:
def hess_explicit(X,G):
    mot = vsr.Mot(*X)
    J = np.array([jaci2(a, mot) for a in points_a]).reshape(-1,6)
    
    mot = vsr.Mot(*X)
    g = mot.rev() * vsr.Mot(*G)
    g = vsr.Dll(g[1], g[2], g[3], g[4], g[5], g[6])
    H = Hess(points_a, points_b, mot)
    B = linear_solver.solve(H,g)
    return np.array(mot * vsr.Dll(*B))
    

In [294]:
def precon_explicit(X,G):
    mot = vsr.Mot(*X)
    g = mot.rev() * vsr.Mot(*G)
    g = vsr.Dll(g[1], g[2], g[3], g[4], g[5], g[6])
#     H = Hess(points_a, points_b, mot)
    J = np.array([jaci2(a, mot) for a in points_a]).reshape(-1,6)
    H = J.T.dot(J)
    B = linear_solver.solve(H,g)
    return np.array(mot * vsr.Dll(*B))

In [357]:
n_points=3

m0 = create_rotor()
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)) for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]

manifold = Sphere(4)
problem = Problem(manifold=manifold, cost=cost, grad=grad_explicit, hess=hess_explicit, verbosity=1)

# (3) Instantiate a Pymanopt solver
# solver = SteepestDescent()
# solver = TrustRegions()
# solver = ParticleSwarm()
# solver = ConjugateGradient()
# solver = NelderMead()


# let Pymanopt do the rest
Xopt = solver.solve(problem, x=np.array([1.,0.,0.,0.]))
print(vsr.Rot(*Xopt))
print(vsr.Rot(*Xopt).rev() * m0)

Compiling cost function...
Optimizing...


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. versor_pybind11.Mot(arg0: float, arg1: float, arg2: float, arg3: float, arg4: float, arg5: float, arg6: float, arg7: float)
    2. versor_pybind11.Mot(arg0: vsr::Multivector<vsr::algebra<vsr::metric<4, 1, true>, double>, vsr::Basis<(short)0, (short)1, (short)2, (short)4, (short)8, (short)16, (short)3, (short)5, (short)6, (short)9, (short)10, (short)12, (short)17, (short)18, (short)20, (short)24, (short)7, (short)11, (short)13, (short)14, (short)19, (short)21, (short)22, (short)25, (short)26, (short)28, (short)15, (short)23, (short)27, (short)29, (short)30, (short)31> >)

Invoked with: 1.0, 0.0, 0.0, 0.0

In [550]:
m0 = create_motor()
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n_points)]
points_b = [point.spin(m0) for point in points_a]

In [551]:
R = create_motor()
A = points_a[0]
B = points_b[0]

In [510]:
def df(R,U,A,B):
    A = vsr.EGA(A)
    B = vsr.EGA(B)
    R = vsr.EGA(R)
    U = vsr.EGA(U)
    X = R * U
    return -(((X * A * R.rev() + R * A * X.rev()) * B))[0]

In [552]:
def df(R,U,A,B):
    A = vsr.CGA(A)
    B = vsr.CGA(B)
    R = vsr.CGA(R)
#     U = vsr.CGA(U)
    X = R * U
    return -(((X * A * R.rev() + R * A * X.rev()) * B))[0]

In [523]:
e12 = vsr.EGA(vsr.Rot(0,1,0,0))
e13 = vsr.EGA(vsr.Rot(0,0,1,0))
e23 = vsr.EGA(vsr.Rot(0,0,0,1))
Y = vsr.Rot(0.0, df(R, e12, A, B), df(R, e13, A, B), df(R, e23, A, B)) * R.rev()

In [553]:
e12 = vsr.CGA(vsr.Mot(0,1,0,0,0,0,0,0))
e13 = vsr.CGA(vsr.Mot(0,0,1,0,0,0,0,0))
e23 = vsr.CGA(vsr.Mot(0,0,0,1,0,0,0,0))
e1i = vsr.CGA(vsr.Mot(0,0,0,0,1,0,0,0))
e2i = vsr.CGA(vsr.Mot(0,0,0,0,0,1,0,0))
e3i = vsr.CGA(vsr.Mot(0,0,0,0,0,0,1,0))
np.array(vsr.Mot(0.0, df(R, e12, A, B), df(R, e13, A, B), df(R, e23, A, B),
                df(R, e1i, A, B), df(R, e2i, A, B), df(R, e3i, A, B), 0.0) * R.rev())

array([ 0.1575, -0.3595,  1.0046,  1.966 , -0.0256,  0.8427, -3.0495,
        1.2663])

In [554]:
gradk3(A,B,np.array(R))

array([-0.1575, -0.3595,  1.0046,  1.966 , -0.0256,  0.8427, -3.0495,
       -1.2663])

In [546]:
def g(R,A,B):
    
    
    
    A = vsr.EGA(A)
    B = vsr.EGA(B)
    R = vsr.EGA(R)
    
    Y = (A * R.rev() * B * R) 
    Y[0] = 0.0 # project onto grade 2
    Y = Y * R.rev() * -2.0
    
    s = vsr.EGA(vsr.Rot(1,0,0,0))
    sr = s.rev()
    e12 = vsr.EGA(vsr.Rot(0,1,0,0))
    e13 = vsr.EGA(vsr.Rot(0,0,1,0))
    e23 = vsr.EGA(vsr.Rot(0,0,0,1))
    e12r = e12.rev()
    e13r = e13.rev()
    e23r = e23.rev()
    
    X0 = sr * (s * Y)[0]
    X1 = e12r * (e12 * Y)[0]
    X2 = e13r * (e13 * Y)[0]
    X3 = e23r * (e23 * Y)[0]
    
    return X0 + X1 + X2 + X3

    

In [559]:
def g2(R,A,B):
    
    
    
    A = vsr.CGA(A)
    B = vsr.CGA(B)
    R = vsr.CGA(R)
    
    Y = (A * R.rev() * B * R) 
    Y[0] = 0.0 # project onto grade 2
    Y[26] = 0.0
    Y[27] = 0.0
    Y[28] = 0.0
    Y[29] = 0.0
    Y[30] = 0.0    

    Y = Y * R.rev() * -2.0
    
    s = vsr.CGA(vsr.Mot(1,0,0,0,0,0,0,0))
    sr = e12r = vsr.CGA(vsr.MotRec(1,0,0,0,0,0,0,0))
    
    e12 =     vsr.CGA(vsr.Mot(0, 1,0,0,0,0,0,0))
    e12r = vsr.CGA(vsr.MotRec(0,-1,0,0,0,0,0,0))
    
    e13 =      vsr.CGA(vsr.Mot(0,0,1,0,0,0,0,0))
    e13r = vsr.CGA(vsr.MotRec(0,0,-1,0,0,0,0,0))
    
    e23 =     vsr.CGA(vsr.Mot(0,0,0, 1,0,0,0,0))
    e23r = vsr.CGA(vsr.MotRec(0,0,0,-1,0,0,0,0))
    
        e1i = vsr.CGA(vsr.Mot(0,0,0,0,1,0,0,0))
    e1ir = vsr.CGA(vsr.MotRec(0,0,0,0,1,0,0,0))
    
    e2i =     vsr.CGA(vsr.Mot(0,0,0,0,0,1,0,0))
    e2ir = vsr.CGA(vsr.MotRec(0,0,0,0,0,1,0,0))
    
    e3i =     vsr.CGA(vsr.Mot(0,0,0,0,0,0,1,0))
    e3ir = vsr.CGA(vsr.MotRec(0,0,0,0,0,0,1,0))
    
    e123i = vsr.CGA(vsr.Mot(0,0,0,0,0,0,0,1))
    e123ir = vsr.CGA(vsr.MotRec(0,0,0,0,0,0,0,-1))


    X0 = sr * (s * Y)[0]
    X1 = e12r * (e12 * Y)[0]
    X2 = e13r * (e13 * Y)[0]
    X3 = e23r * (e23 * Y)[0]
    X4 = e1ir * (e1i * Y)[0]
    X5 = e2ir * (e2i * Y)[0]
    X6 = e3ir * (e3i * Y)[0]
    X7 = e123ir * (e123i * Y)[0]
    
    return X0 + X1 + X2 + X3 + X4 + X5 + X6 + X7

In [572]:
G = g2(R,A,B)
print(G)

CGA: [ -0.88 0 0 0 0 0 0.72 -1.2 -1.2 -0.25 0.2 -2.6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1.5 0 0 0 0 0 ]


In [573]:
gradk3(A,B,np.array(R))

array([-0.1575, -0.3595,  1.0046,  1.966 , -0.0256,  0.8427, -3.0495,
       -1.2663])

In [583]:
U = vsr.CGA(vsr.Mot(0,1,2,3,4,5,6,0))

In [584]:
(G * (vsr.CGA(R) * U))[0]

-15.214316598325281

In [585]:
df(R,U,A,B)

-15.214316598325283

In [385]:
solver = TrustRegions

In [ ]:
solver = TrustRegions

In [ ]:
r = create_rotor()
grad_explicit(r)

In [214]:
Y  = R.rev() * vsr.Biv(*np.sum([np.array(b ^ a.spin(R)) for a,b in zip(points_a, points_b)],axis=1))
Y

Rot: [ -0.69 -1.8 -1.2 0.6 ]

In [215]:
manifold.proj(np.array(R), np.array(Y))

array([ 0.2791, -1.6261, -0.6981,  0.9759])

In [358]:
manifold = Sphere(4)
problem = Problem(manifold=manifold, cost=cost, verbosity=1)

# (3) Instantiate a Pymanopt solver
# solver = SteepestDescent()
# solver = TrustRegions()
# solver = ParticleSwarm()
solver = ConjugateGradient()
# solver = NelderMead()


# let Pymanopt do the rest
Xopt = solver.solve(problem, x=np.array([1.,0.,0.,0.]))
print(vsr.Rot(*Xopt))
print(vsr.Rot(*Xopt).rev() * m0)

Compiling cost function...
Computing gradient of cost function...
Optimizing...


ValueError: too many values to unpack (expected 3)

In [217]:
Rtest = create_rotor()


In [60]:
grad_explicit(np.array(Rtest))

array([-1.6031, -4.8323,  2.0965, -0.8448])

In [61]:
np.dot(np.array(R).T, G)

ValueError: shapes (4,) and (8,) not aligned: 4 (dim 0) != 8 (dim 0)

In [62]:
Y = vsr.EGA(*np.sum([np.array(vsr.EGA(a) * vsr.EGA(Rtest).rev() * vsr.EGA(b) +
 vsr.EGA(a).rev() * vsr.EGA(Rtest.rev()) * vsr.EGA(b).rev()) for a, b in zip(points_a, points_b)], axis=0))

In [63]:
Y_ = Y * vsr.EGA(Rtest)
Y_[0] = 0
G = Y_ * vsr.EGA(Rtest.rev()) * -2
# G[1] * -1
# G[2] * -1
# G[3] * -1

In [64]:
np.array(G.rev())

array([-1.6031, -0.    , -0.    , -0.    , -4.8323,  2.0965, -0.8448,  0.    ])

In [66]:
np.dot(np.array(G.rev()), np.array(vsr.EGA(Rtest)))

-2.7755575615628914e-17